# 问题一

问题描述：编写一个程序：get_response(saying, response_rules)。输入是一个字符串和我们定义的 rules，输出是一个回答。

In [10]:
# 英文最终版
from collections import defaultdict
fail = [True, None]

def is_variable(pat):
    return(pat.startswith('?') and all(s.isalpha() for s in pat[1:]))

def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

# 兼容原本的 pat_to_dict 函数
def pat_to_dict_imp(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)


def subsitite(rule, parsed_rules):
    if not rule: return []
    
    # Python 字典(Dictionary) get() 函数返回指定键的值，如果值不在字典中返回默认值。语法：dict.get(key, default=None)
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

def pat_match_with_seg(pattern, saying):
    
    if len(pattern) != 0 and not saying: return fail
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    

    
def get_response(saying, response_rules):
    """" please implement the code, to get the response as followings:   
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    import random
    
    for pattern in response_rules:
        got_pattern = pat_match_with_seg( pattern.split(), saying.split() )
        
        if got_pattern[-1] == None:
            pass
        else:
            pattern_corr_baselen = len( response_rules[pattern] )
            resp = subsitite( response_rules[pattern][ random.randint(0, pattern_corr_baselen - 1) ].split(), pat_to_dict_imp(got_pattern))
            return( " ".join(resp) )
    return("Sorry, I can't understand.")

## 例子

In [15]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['What would it mean if you got ?y ？', 'Why do you want ?y ？', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['Why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']}

get_response("Hi, I was sad",rule_responses)

'Perhaps I already knew you were sad'

# 问题二

问题描述：改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词。

In [ ]:
# 中文版
from collections import defaultdict
fail = [True, None]

def is_variable(pat):
    return(pat.startswith('?') and all(s.isalpha() for s in pat[1:]))

def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

# 兼容原本的 pat_to_dict 函数
def pat_to_dict_imp(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)


def subsitite(rule, parsed_rules):
    if not rule: return []
    
    # Python 字典(Dictionary) get() 函数返回指定键的值，如果值不在字典中返回默认值。语法：dict.get(key, default=None)
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

# 自定义函数，中文环境下 pattern 匹配
def join_pattern(test):
    A = []
    i = 0
    while i < len(test):   
        if test[i] == "?":
            if test[i + 1] == "x" or test[i + 1] == "y" or test[i + 1] == "z":
                A.append("".join([test[i], test[ i + 1 ]]))
                i += 2
            elif test[i + 1] == "*":
                A.append("".join([test[i], test[ i + 1 ], test[ i + 2 ]]))
                i += 3
            else:
                A.append(test[i])
                i += 1
        else:
            A.append(test[i])
            i += 1
    return(A)


def pat_match_with_seg(pattern, saying):
    
    if len(pattern) != 0 and not saying: return fail
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    

def get_response(saying, response_rules):
    """" please implement the code, to get the response as followings:   
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    import random
    
    for pattern in response_rules:
        got_pattern = pat_match_with_seg( join_pattern(list(jieba.cut(pattern, cut_all=False))), list(jieba.cut(saying, cut_all=False)) )
        
        if got_pattern[-1] == None:
            pass
        else:
            pattern_corr_baselen = len( response_rules[pattern] )
            resp = subsitite( join_pattern( list( jieba.cut(response_rules[pattern][ random.randint(0, pattern_corr_baselen - 1) ]))) , pat_to_dict_imp(got_pattern))
            return( "".join(resp) )
    return("对不起，我听不懂。")

## 例子

我在实验室电脑上已经成功运行了，但是不知道为什么无法安装 jieba 分词，我先提交这个版本，4月8日我会再提供一个确保成功的版本，不好意思！

# 问题三

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式。

# 问题四

1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？

程序的优点是可以针对套路性的问题进行针对性的回答，包括课堂讲授的海底捞的模式，改进后我觉得完全可以回应。而且，在词库中的问题可以回答十分准确。但是缺点也很明显，第一，缺少灵活性和应变性，人的回答主要就体现在这一点上，人是根据自己的情况进行回答，这个提供的模式十分有限。其次，对于不在模式之中的问题无能为力，只能回应没有听懂了。第三，没有对于词语意义的分析，正常人的说话不同的话可能具有相同的意义，程序不能胜任。第四，有限性，相对于人来说，人是存在意义上的无限者，体现在语言上也是无限者，程序本质上是有限的。

2.什么是数据驱动？数据驱动在这个程序里如何体现？

我之前没有听过这个词汇，从这个程序上说一下字面的理解：数据驱动就是让数据自己说话，不那么人为地做决定。我们也看到了，少量的数据产生单调的回答，如果有更多真人的回答，效果肯定更加地好。我们程序中的体验就是 response_rules 的丰富程度决定性的影响了对话体验。

3.数据驱动与 AI 的关系是什么？

如果我让我写的程序接受图灵测试，估计很快就露馅了。那么，我们假设地狠一点，我们的数据中包含了这个民族历史上所有的回答。那么它能受得住图灵测试吗？我看答案还是否定的，因为它缺乏一个内在的一致性，我们人来说，作为一个个体是具有一贯性的。所以，有一个两难的情况就是，数据太多，则会天上一句，地上一句；数据太少则会单调。可能在数据的采集上，采集一个人的一生所言更为合适了。当然，也有模型的原因，我在这里做的这个，每一天的选取在随机的，可能提供一些好的分布模型效果也会更好。如果提供好的时间序列，也会有更好的回答。在这个意义上，数据驱动确实给了我们一个 AI 方向的进路。不过，如果我们尝试这样问自己，我是数据驱动的吗？我觉得少有人觉得自己是数据驱动的，原因可能因为我们在对话中具有情感，夹杂着我们的经验，有一时一地的偶然性还有很多其他因素。所以，即便一个人曾经说的话都知道，对于他将要说的话的预测，至少现有的这个模型还是不行。